In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from scipy.stats import pearsonr
from ase import Atoms

In [11]:
# json file read. (MMFFstructure optimization, Modelstructure optimization)
# check the optimization converges to the same structure.
# then 
# 1) check the number of optimization steps between MMFF and Model.
# 2) check the number of SCF steps between MMFF and Model.
# 3) check the total cpu time between MMFF and Model.



In [39]:
json_file = "/home/share/DATA/NeuralOpt/QM9M_results/save_xyz.morse.128steps/results/randomly_select/initial/calculation_log/idx110/input.json"
log_file = "/home/share/DATA/NeuralOpt/QM9M_results/save_xyz.morse.128steps/results/randomly_select/initial/calculation_log/idx110/input.log"

def read_log(json_file, log_file):
    import json
    with open(json_file) as f:
        json_data = json.load(f)
    # check json file
    opt_done = json_data["optimization"]["done"]
    print(json_data["optimization"])
    scf_energies = json_data["optimization"]["scf energies"]
    xyz = np.array(json_data["atoms"]["coords"]["3d"]).reshape(-1, 3)
    symb = json_data["atoms"]["elements"]["number"]
    atoms = Atoms(symbols=symb, positions=xyz)
    
    data = {"opt_done": opt_done, "scf_energies": scf_energies, "atoms": atoms}
    
    # check log file
    with open(log_file) as f:
        lines = f.readlines()
    # the index of SCF info line
    line_index = [i for i, line in enumerate(lines) if "SCF Done" in line]
    # number of scf cycles
    scf_cycles = []
    scf_cpu_times = []
    for i in line_index:
        line = lines[i]
        comp = line.strip().split()
        assert comp[-1] == "cycles"
        assert lines[i+3].strip().split()[-2] == "cpu:"
        
        n_cycle = int(comp[-2])
        cpu_time = float(lines[i+3].strip().split()[-1])
        scf_cycles.append(n_cycle)
        scf_cpu_times.append(cpu_time)
        
    line = lines[-3]
    # line = "Job cpu time:       0 days  0 hours 33 minutes  0.7 seconds."
    total_cpu_time = line.split()[-4:]
    total_cpu_time = [int(x) for x in total_cpu_time if x.isdigit()]
    assert len(total_cpu_time) == 4
    total_cpu_time = total_cpu_time[0]*24*60 + total_cpu_time[1]*60 + total_cpu_time[2] + total_cpu_time[3]/60
    
    data["scf_cycles"] = scf_cycles
    data["scf_cpu_times"] = scf_cpu_times
    data["total_cpu_time"] = total_cpu_time
    return data

In [40]:
data = read_log(json_file, log_file)

KeyError: 'scf energies'

<Popen: returncode: None args: ['/home/ksh/.conda/envs/NeuralOpt/bin/python'...>

usage: ase [-h] [--version] [-T]
           {help,info,test,gui,db,run,band-structure,build,dimensionality,eos,ulm,find,nebplot,nomad-upload,nomad-get,convert,reciprocal,completion,diff,exec}
           ...
ase: error: TclError: no display name and no $DISPLAY environment variable
To get a full traceback, use: ase -T gui ...
